# Plateforme Agnostique de Traitement et d'Analyse des Textes
### Carnet d'expérimentation
---

## Sujet : Pipeline classif infox supervisé sur recueil

---

# Observations et environnement
---

In [1]:
random_state = 42

In [2]:
cd ../..

/Volumes/Geek/Work/Patat


In [3]:
import ast
import importlib
import pandas as pd
import seaborn as sns

In [4]:
filename = 'data/tmp/230508-RecueilFiltre.csv'

In [5]:
df_recueil = pd.read_csv(filename)
df_recueil['paragraphs'] = df_recueil['paragraphs'].apply(ast.literal_eval)
df_recueil['abstract'].fillna('',inplace=True)

In [9]:
import patat.imp.importer

In [10]:
imp = patat.imp.importer.Importer()

In [11]:
df_recueil['site'] = df_recueil['url'].apply(imp.get_site)

In [31]:
df_recueil[df_recueil['tds']==1]

,#site,# article,Site,url,Auteur/trice,Date,Titre,explication,infox,signe,...,date,tags,paragraphs,date_iso,abstract,c_count,p_count,p_size,site,text
0,1,1,Alnas,https://www.alnas.fr/actualite/communaute/lara...,NaN,31/10/2022,L’Arabie saoudite organise des célébrations d'...,NaN,0,0.0,...,31/10/2022,['ActualitésCommunauté'],[L’Arabie saoudite a organisé un événement pou...,2022-10-31T00:00:00,,2277,8,284.625000,www.alnas.fr,L’Arabie saoudite organise des célébrations d'...
1,1,2,Alnas,https://www.alnas.fr/actualite/en-vrac/quel-ar...,NaN,19/10/2022,« Quel art ! » : un étudiant en droit invente ...,NaN,0,0.0,...,19/10/2022,['ActualitésEn vrac'],[Une professeure de droit a partagé sur Twitte...,2022-10-19T00:00:00,,1683,8,210.375000,www.alnas.fr,« Quel art ! » : un étudiant en droit invente ...
2,1,3,Alnas,https://www.alnas.fr/actualite/politique/laust...,NaN,18/10/2022,L’Australie ne reconnaît plus Jérusalem comme ...,NaN,0,0.0,...,18/10/2022,['ActualitésPolitique'],[Le gouvernement israélien a qualifié la posit...,2022-10-18T00:00:00,,2678,15,178.533333,www.alnas.fr,L’Australie ne reconnaît plus Jérusalem comme ...
3,1,4,Alnas,https://www.alnas.fr/actualite/communaute/nant...,NaN,17/10/2022,Nantes : une mère musulmane meurt poignardée e...,NaN,0,0.0,...,17/10/2022,['ActualitésCommunauté'],[NANTES – Une enquête pour « homicide volontai...,2022-10-17T00:00:00,,1792,5,358.400000,www.alnas.fr,Nantes : une mère musulmane meurt poignardée e...
4,1,5,Alnas,https://www.alnas.fr/actualite/en-vrac/la-fran...,NaN,10/11/2022,La France condamnée pour avoir placé un enfant...,le titre est décalé (donne une impression faus...,0,0.0,...,10/11/2022,['ActualitésEn vrac'],[La Cour européenne des droits de l’Homme (CED...,2022-11-10T00:00:00,,1482,5,296.400000,www.alnas.fr,La France condamnée pour avoir placé un enfant...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,44,617,Wikistrike,https://www.wikistrike.com/2022/10/selon-un-an...,NaN,04/10/2022,"Selon un ancien conseiller du Pentagone, les É...",colporte les ragots (vrais ou pas) d'un ex con...,1,0.0,...,Publié par wikistrike.com\n ...,[],"[Selon un ancien conseiller du Pentagone, les ...",2022-10-04T00:00:00,,3976,14,284.000000,www.wikistrike.com,"Selon un ancien conseiller du Pentagone, les É..."
624,44,618,Wikistrike,https://www.wikistrike.com/2022/10/la-pologne-...,NaN,31/10/2022,La Pologne s’apprête à s’emparer du territoire...,relai de ragots invérifiés,1,0.0,...,Publié par wikistrike.com\n ...,[],[La Pologne se prépare à s’emparer du territoi...,2022-10-31T00:00:00,,2284,8,285.500000,www.wikistrike.com,La Pologne s’apprête à s’emparer du territoire...
625,44,619,Wikistrike,https://www.wikistrike.com/2022/11/exclusivite...,NaN,19/11/2022,Exclusivité WikiStrike: un physicien en colère...,délire total d'un physicien anonyme (ahurissan...,1,0.0,...,Publié par wikistrike.com\n ...,[],"[Crise énergétique battant son plein, WikiStr...",2022-11-19T00:00:00,,27223,27,1008.259259,www.wikistrike.com,Exclusivité WikiStrike: un physicien en colère...
626,44,620,Wikistrike,https://www.wikistrike.com/2022/12/zelensky-pa...,NaN,23/12/2023,Zelensky parti demander à papa Biden une aide ...,l'article est vide (même si le titre accuse Ze...,1,0.0,...,Publié par wikistrike.com\n ...,[],"[Alors que des milliers d’Ukrainiens meurent, ...",2022-12-23T00:00:00,,557,5,111.400000,www.wikistrike.com,Zelensky parti demander à papa Biden une aide ...


# Experience
---

## Textes

In [23]:
def get_text(row):
    text = row['title']+'\n\n'
    if row['abstract']:
         text = text + row['abstract']+'\n\n'
    text = text + '\n\n'.join(row['paragraphs'])
    return text

In [24]:
df_recueil['text'] = df_recueil.apply(get_text,axis=1)

In [26]:
for text in df_recueil['text'].sample(1):
    print(text)

Burkini dans les piscines municipales : Pour Darmanin, le choix fait à Grenoble est une « inacceptable provocation communautaire »

Source : 20minutes

La polémique continue d’enfler autour du burkini. Gérald Darmanin a qualifié mardi d'« inacceptable provocation communautaire » l’autorisation du port de ce maillot de bain dans les piscines municipales de Grenoble.

Le conseil municipal de Grenoble, dirigé par le maire écologiste Eric Piolle, a en effet validé lundi par une courte majorité une modification du règlement intérieur des piscines se traduisant par l’autorisation du port du burkini.

« M. Piolle, soutien de M. Mélenchon, joue l’inacceptable provocation communautaire, contraire à nos valeurs », a tweeté le ministre de l’Intérieur du gouvernement démissionnaire. « J’ai donné instruction au préfet de déférer en "déféré laïcité" la délibération permettant le port du "Burkini" et, le cas échéant, d’en demander le retrait », a-t-il ajouté. Le déféré préfectoral s’inscrit dans le c

## Ignore Words / Lemmas

In [32]:
import patat.util.text
import patat.ml.lex_analyser
lex = patat.ml.lex_analyser.LexAnalyser()

### Lemmas

In [74]:
preprocessor = patat.util.text.preprocess
tokenizer = lex.get_lemmas_from_text

In [35]:
df_words = lex.get_df_words(texts= df_recueil['text'],values = df_recueil['infox'].astype(int), preprocessor=preprocessor, tokenizer= tokenizer)
### Rare Words
for occ_rare in [1,2,3,4,5,6,7]:
    df_words['rare_'+str(occ_rare)]=(df_words['count_0']<=occ_rare)&(df_words['count_1']<=occ_rare)
### Common words
for common_size in [50,80,120,200,500]:
    common_0 = df_words.sort_values('count_0',ascending=False).head(common_size).index
    common_1 = df_words.sort_values('count_1',ascending=False).head(common_size).index
    df_words['common_'+str(common_size)]=(df_words.index.isin(common_0))&(df_words.index.isin(common_1))
### Ignore Lemmas
ignore_lemmas={
    'None' : None,
    'small' : list(df_words[df_words['common_80']].index)+list(df_words[df_words['rare_1']].index),
    'medium' : list(df_words[df_words['common_80']].index)+list(df_words[df_words['rare_2']].index),
    'large' : list(df_words[df_words['common_200']].index)+list(df_words[df_words['rare_5']].index),
    'xlarge' : list(df_words[df_words['common_500']].index)+list(df_words[df_words['rare_7']].index),
}

/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [36]:
df_words

,count_0,count_1,count,rare_1,rare_2,rare_3,rare_4,rare_5,rare_6,rare_7,common_50,common_80,common_120,common_200,common_500
de,16053,22318,38371,False,False,False,False,False,False,False,True,True,True,True,True
la,9058,12076,21134,False,False,False,False,False,False,False,True,True,True,True,True
le,6711,9591,16302,False,False,False,False,False,False,False,True,True,True,True,True
et,6384,9454,15838,False,False,False,False,False,False,False,True,True,True,True,True
les,6110,9761,15871,False,False,False,False,False,False,False,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
iut,0,1,1,True,True,True,True,True,True,True,False,False,False,False,False
ivano,0,1,1,True,True,True,True,True,True,True,False,False,False,False,False
ivermectin,0,2,2,False,True,True,True,True,True,True,False,False,False,False,False
iversen,0,18,18,False,False,False,False,False,False,False,False,False,False,False,False


In [45]:
ignore_lemmas['large']

['de',
 'la',
 'le',
 'et',
 'les',
 'des',
 'en',
 'un',
 'est',
 'que',
 'du',
 'une',
 'avoir',
 'être',
 'pour',
 'qui',
 'dans',
 'il',
 'par',
 'pas',
 'sur',
 'ce',
 'au',
 'plaire',
 'faire',
 'ne',
 'nous',
 'qu',
 'pouvoir',
 'se',
 'avec',
 'mais',
 'tout',
 'ou',
 'son',
 'été',
 'aux',
 'vous',
 'comme',
 'cette',
 'ces',
 'on',
 'même',
 'elle',
 'devoir',
 'autre',
 'pays',
 'leur',
 'si',
 'je',
 'russie',
 'ses',
 'ils',
 'aussi',
 'état',
 'sa',
 'aller',
 'bien',
 'grand',
 'politique',
 'sans',
 'dire',
 'contre',
 'france',
 'français',
 'voir',
 'russe',
 'premier',
 'monde',
 'nouveau',
 'an',
 'depuis',
 'entre',
 'alors',
 'mettre',
 'guerre',
 'encore',
 'après',
 'deux',
 'uni',
 'europe',
 'année',
 'ukraine',
 'cela',
 'dernier',
 'temps',
 'européen',
 'non',
 'leurs',
 'selon',
 'prendre',
 'tous',
 'moins',
 'personne',
 'dont',
 'déclarer',
 'gouvernement',
 'présider',
 'certain',
 'lui',
 'car',
 'ainsi',
 'où',
 'covid',
 'très',
 'donner',
 'sou',
 

In [54]:
sw_text = ' '.join(ignore_lemmas['small'])

In [55]:
len(tokenizer(sw_text))

12270

In [57]:
len(ignore_lemmas['small'])

12270

In [120]:
len(set(ignore_lemmas['small'])-set(tokenizer(sw_text)))

37

In [51]:
tokenizer('équivaloir')

['équivaloir']

## Pipeline

In [38]:
from sklearn import metrics
from sklearn.model_selection import cross_validate, train_test_split

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

tokenizer = lex.get_lemmas_from_text

logreg = LogisticRegression(C=1, random_state=random_state, solver='lbfgs', multi_class='ovr', max_iter=1000)
neuralnet = MLPClassifier(alpha=1, max_iter=1000, hidden_layer_sizes=(8,8), random_state=random_state)

pipelineLR = Pipeline(steps = [
    ('vectorizer',TfidfVectorizer(preprocessor=preprocessor, tokenizer=tokenizer, stop_words=ignore_lemmas['large'])), 
    ('classifier',logreg)
                  ])

pipelineMLPC = Pipeline(steps = [
    ('vectorizer',TfidfVectorizer(preprocessor=preprocessor, tokenizer=tokenizer, stop_words=ignore_lemmas['large'])), 
    ('classifier',neuralnet)
                  ])

train_size = 0.80
X_train, X_test, y_train, y_test = train_test_split(df_recueil['text'], df_recueil['infox'], train_size=train_size, random_state=random_state)

for pipeline in [pipelineLR,pipelineMPLC]:
    pipeline.fit(X_train, y_train)

    y_preds = pipeline.predict(X_test)
    y_probas = pipeline.predict_proba(X_test)

    results = {}
    results['roc_auc'] = metrics.roc_auc_score(y_test, y_probas[:, 1])
    results['f1']=metrics.f1_score(y_test, y_preds)
    results['accuracy']=metrics.accuracy_score(y_test, y_preds)
    results['recall']=metrics.recall_score(y_test, y_preds)
    results['log_loss'] = metrics.log_loss(y_test, y_preds)
    cnf_matrix = metrics.confusion_matrix(y_test, y_preds,normalize='all')
    results['false_positive']=cnf_matrix[0,1]
    results['false_negative']=cnf_matrix[1,0]

    print(results)

/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aboutir', 'affilier', 'alambiquer', 'ambitieux', 'boire', 'bouillir', 'climatiser', 'commercer', 'coordonner', 'croire', 'divertir', 'décombrer', 'décrépir', 'déplaire', 'désordonner', 'florir', 'grandir', 'inclure', 'intégrer', 'joindre', 'matcher', 'maudire', 'menacer', 'mol', 'mou', 'nuire', 'obéir', 'oxygéner', 'pondérer', 'presser', 'précieux', 'présumer', 'recroître', 'saisir', 'satisfaire', 'standardiser', 'structurer', 'subventionner', 'suivre', 'tracter', 'urger', 'vieillir', 'vrombir', 'émerger', 'énoncer', 'équivaloir'] no

{'roc_auc': 0.8914748292436125, 'f1': 0.7777777777777777, 'accuracy': 0.8095238095238095, 'recall': 0.711864406779661, 'log_loss': 6.865457788403268, 'false_positive': 0.05555555555555555, 'false_negative': 0.1349206349206349}


/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aboutir', 'affilier', 'alambiquer', 'ambitieux', 'boire', 'bouillir', 'climatiser', 'commercer', 'coordonner', 'croire', 'divertir', 'décombrer', 'décrépir', 'déplaire', 'désordonner', 'florir', 'grandir', 'inclure', 'intégrer', 'joindre', 'matcher', 'maudire', 'menacer', 'mol', 'mou', 'nuire', 'obéir', 'oxygéner', 'pondérer', 'presser', 'précieux', 'présumer', 'recroître', 'saisir', 'satisfaire', 'standardiser', 'structurer', 'subventionner', 'suivre', 'tracter', 'urger', 'vieillir', 'vrombir', 'émerger', 'énoncer', 'équivaloir'] no

{'roc_auc': 0.9028585884138629, 'f1': 0.8173913043478261, 'accuracy': 0.8333333333333334, 'recall': 0.7966101694915254, 'log_loss': 6.007275564852859, 'false_positive': 0.07142857142857142, 'false_negative': 0.09523809523809523}


## Crossvalidation

In [115]:
from sklearn.model_selection import cross_val_score,cross_validate
X = df_recueil['text']
y = df_recueil['infox']

In [ ]:
pipelineLR = Pipeline(steps = [
    ('vectorizer',TfidfVectorizer(preprocessor=preprocessor, tokenizer=tokenizer, stop_words=ignore_lemmas['small'])), 
    ('classifier',logreg)
                  ])

pipelineMLPC = Pipeline(steps = [
    ('vectorizer',TfidfVectorizer(preprocessor=preprocessor, tokenizer=None, stop_words=None)), 
    ('classifier',neuralnet)
                  ])

In [117]:
classifier = pipelineLR
scores = cross_validate(classifier, X, y, cv=5,scoring=('roc_auc','f1','accuracy','recall'))
scores

/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aboutir', 'alambiquer', 'ambitieux', 'bouillir', 'breveter', 'bruire', 'bénir', 'climatiser', 'cuire', 'divertir', 'décrépir', 'déplaire', 'désordonner', 'instruire', 'matcher', 'maudire', 'mol', 'mou', 'muqueux', 'obéir', 'oxygéner', 'pondérer', 'saisir', 'satisfaire', 'spécifier', 'subventionner', 'vieillir', 'vrombir'] not in stop_words.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used si

{'fit_time': array([0.36175489, 0.34003305, 0.36613297, 0.38266587, 0.33973193]),
 'score_time': array([0.16125393, 0.12684703, 0.17539096, 0.14286113, 0.14137077]),
 'test_roc_auc': array([0.76590909, 0.83333333, 0.9270202 , 0.63020031, 0.64997432]),
 'test_f1': array([0.6       , 0.79646018, 0.85483871, 0.53571429, 0.64      ]),
 'test_accuracy': array([0.71428571, 0.81746032, 0.85714286, 0.584     , 0.64      ]),
 'test_recall': array([0.45      , 0.75      , 0.88333333, 0.50847458, 0.6779661 ])}

## Entrainement sur tout le dataset

In [40]:
pipeline = Pipeline(steps = [
    ('vectorizer',TfidfVectorizer(preprocessor=preprocessor, tokenizer=tokenizer, stop_words=ignore_lemmas['large'])), 
    ('classifier',logreg)
                  ])

pipeline.fit(df_recueil['text'], df_recueil['infox'])


/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/p311/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aboutir', 'affilier', 'alambiquer', 'ambitieux', 'boire', 'bouillir', 'climatiser', 'commercer', 'coordonner', 'croire', 'divertir', 'décombrer', 'décrépir', 'déplaire', 'désordonner', 'florir', 'grandir', 'inclure', 'intégrer', 'joindre', 'matcher', 'maudire', 'menacer', 'mol', 'mou', 'nuire', 'obéir', 'oxygéner', 'pondérer', 'presser', 'précieux', 'présumer', 'recroître', 'saisir', 'satisfaire', 'standardiser', 'structurer', 'subventionner', 'suivre', 'tracter', 'urger', 'vieillir', 'vrombir', 'émerger', 'énoncer', 'équivaloir'] no

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(preprocessor=<function preprocess at 0x146f28180>,
                                 stop_words=['de', 'la', 'le', 'et', 'les',
                                             'des', 'en', 'un', 'est', 'que',
                                             'du', 'une', 'avoir', 'être',
                                             'pour', 'qui', 'dans', 'il', 'par',
                                             'pas', 'sur', 'ce', 'au', 'plaire',
                                             'faire', 'ne', 'nous', 'qu',
                                             'pouvoir', 'se', ...],
                                 tokenizer=<bound method LexAnalyser.get_lemmas_from_text of <patat.ml.lex_analyser.LexAnalyser object at 0x1629e9690>>)),
                ('classifier',
                 LogisticRegression(C=1, max_iter=1000, multi_class='ovr',
                                    random_state=42))])

# Sauvegarde des résultats
---

In [ ]:
import patat.util.file

In [ ]:
patat.util.file.pickle_save(pipelineLR,'data/tmp/230425-LogRegInfox.pickle')

In [ ]:
patat.util.file.pickle_save(pipelineMPLC,'data/tmp/230425-MPLCInfox.pickle')

# Conclusions
---

# Bricolages
---